# Donor Compass Colab Demo (Standalone)

This notebook is a standalone snapshot demo of the Donor Compass allocation library.

## What this notebook includes
- Core calculator and allocation functions
- All aggregation methods currently supported in `donor_compass.py`
- A simple demo runner for one method or all methods
- Two plain-language validation checks:
  - **Check 1: Dominant Then Saturating Project**
  - **Check 2: Same Credences Across Methods**

## Quick start for first-time users
1. Run the dependency/setup cell.
2. Run the standalone implementation cell.
3. Run the single-method demo cells.
4. Run the all-method comparison cell.
5. Run the validation checks cell at the end.

## Important notes
- This notebook is intentionally self-contained (no local file imports required).
- Source of truth is still the Python files in the repository (`donor_compass.py`, `met_sim_utils.py`, `multi_stage_aggregation.py`).
- If running in Colab, run cells from top to bottom (or use **Runtime -> Run all**).

In [7]:
# Colab dependency setup
# This installs required packages only when running in Colab.
import sys
import subprocess

if "google.colab" in sys.modules:
    print("Running in Colab - installing dependencies...")
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "-q",
            "numpy",
            "scipy",
            "scikit-learn",
            "pandas",
        ]
    )
else:
    print("Not running in Colab - skipping install cell.")

Not running in Colab - skipping install cell.


In [8]:
# Standalone Donor Compass implementation snapshot
# Copied/adapted from donor_compass.py, met_sim_utils.py, and multi_stage_aggregation.py

import random
from dataclasses import dataclass
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr, spearmanr
from sklearn.manifold import MDS


# -----------------------------------------------------------------------------
# Demo data presets
# -----------------------------------------------------------------------------
DEFAULT_PROJECT_DATA = {
    "project_human": {
        "tags": {"near_term_xrisk": False},
        "diminishing_returns": [
            1.0, 0.8, 0.6, 0.45, 0.35, 0.28, 0.22, 0.18, 0.15, 0.12,
            0.10, 0.09, 0.08, 0.07, 0.06,
        ],
        "effects": {
            "effect_human": {
                "recipient_type": "human_life_years",
                "values": [
                    [220, 240, 180, 200],
                    [200, 220, 170, 185],
                    [180, 195, 160, 170],
                    [150, 160, 140, 145],
                    [100, 110, 90, 95],
                    [80, 85, 70, 75],
                ],
            }
        },
    },
    "project_chicken": {
        "tags": {"near_term_xrisk": False},
        "diminishing_returns": [
            1.0, 0.95, 0.90, 0.85, 0.80, 0.75, 0.70, 0.65, 0.60, 0.55,
            0.50, 0.46, 0.42, 0.38, 0.34,
        ],
        "effects": {
            "effect_chicken": {
                "recipient_type": "chickens_birds",
                "values": [
                    [180, 190, 150, 170],
                    [170, 180, 145, 160],
                    [160, 170, 140, 155],
                    [140, 150, 130, 140],
                    [120, 130, 110, 120],
                    [100, 110, 95, 100],
                ],
            }
        },
    },
    "project_fish": {
        "tags": {"near_term_xrisk": False},
        "diminishing_returns": [
            1.0, 0.96, 0.92, 0.88, 0.84, 0.80, 0.76, 0.72, 0.68, 0.64,
            0.60, 0.56, 0.52, 0.48, 0.44,
        ],
        "effects": {
            "effect_fish": {
                "recipient_type": "fish",
                "values": [
                    [140, 150, 120, 130],
                    [135, 145, 115, 125],
                    [130, 140, 110, 120],
                    [120, 130, 105, 115],
                    [110, 120, 95, 105],
                    [100, 110, 90, 100],
                ],
            }
        },
    },
}

EXAMPLE_CUSTOM_WORLDVIEWS = [
    {
        "name": "HumanAnchor",
        "credence": 0.5,
        "moral_weights": {
            "human_life_years": 1.0,
            "human_ylds": 0.4,
            "human_income_doublings": 0.2,
            "chickens_birds": 0.1,
            "fish": 0.05,
            "shrimp": 0.0,
            "non_shrimp_invertebrates": 0.0,
            "mammals": 0.05,
        },
        "discount_factors": [1.0, 0.95, 0.8, 0.6, 0.4, 0.3],
        "risk_profile": 0,
        "p_extinction": 0.05,
    },
    {
        "name": "ChickenAnchor",
        "credence": 0.3,
        "moral_weights": {
            "human_life_years": 0.1,
            "human_ylds": 0.2,
            "human_income_doublings": 0.1,
            "chickens_birds": 1.0,
            "fish": 0.2,
            "shrimp": 0.0,
            "non_shrimp_invertebrates": 0.0,
            "mammals": 0.2,
        },
        "discount_factors": [1.0, 0.95, 0.8, 0.6, 0.4, 0.3],
        "risk_profile": 0,
        "p_extinction": 0.05,
    },
    {
        "name": "FishAnchor",
        "credence": 0.2,
        "moral_weights": {
            "human_life_years": 0.1,
            "human_ylds": 0.2,
            "human_income_doublings": 0.1,
            "chickens_birds": 0.1,
            "fish": 1.0,
            "shrimp": 0.0,
            "non_shrimp_invertebrates": 0.0,
            "mammals": 0.1,
        },
        "discount_factors": [1.0, 0.95, 0.8, 0.6, 0.4, 0.3],
        "risk_profile": 0,
        "p_extinction": 0.05,
    },
]

INCREMENT_SIZE = 10
AGGREGATION_DEFAULTS = {
    "met_threshold": 0.50,
    "nash_disagreement_point": "zero_spending",
    "msa_permissibility_mode": "winner_take_all",
    "msa_top_k": 2,
    "msa_within_percent": 0.10,
    "msa_binary_threshold": 0.0,
    "tie_break": "deterministic",
}
MSA_DEFAULT_BINARY_WORLDVIEWS = {"Kantianism", "Rawlsian Contractarianism"}


# -----------------------------------------------------------------------------
# Calculator functions
# -----------------------------------------------------------------------------
def calculate_single_effect(effect_data, moral_weight, discount_factors, risk_profile):
    values_matrix = np.array(effect_data["values"], dtype=float)
    r_etq = values_matrix[:, int(risk_profile)]
    D_t = np.array(discount_factors, dtype=float)
    return float(moral_weight * np.sum(r_etq * D_t))


def calculate_project(project_data, moral_weights, discount_factors, risk_profile):
    total = 0.0
    breakdown = {}
    for effect_id, effect_data in project_data["effects"].items():
        m_i = moral_weights.get(effect_data["recipient_type"], 0.0)
        value = calculate_single_effect(effect_data, m_i, discount_factors, risk_profile)
        breakdown[effect_id] = value
        total += value
    return {"total": float(total), "breakdown": breakdown}


def calculate_all_projects(data, moral_weights, discount_factors, risk_profile):
    results = {}
    for project_id, project_data in data.items():
        results[project_id] = calculate_project(
            project_data, moral_weights, discount_factors, risk_profile
        )["total"]
    return results


def adjust_for_extinction_risk(project_values, data, p_extinction):
    adjusted = {}
    for project_id, value in project_values.items():
        if data[project_id]["tags"]["near_term_xrisk"]:
            adjusted[project_id] = value
        else:
            adjusted[project_id] = value * (1 - p_extinction)
    return adjusted


def get_diminishing_returns_factor(data, project_id, current_funding):
    step_size = 10.0
    steps = float(current_funding) / step_size
    nearest_step = int(round(steps))
    if np.isclose(steps, nearest_step, atol=1e-9):
        idx = nearest_step
    else:
        idx = int(np.floor(steps))
    idx = max(idx, 0)

    dr_array = data[project_id]["diminishing_returns"]
    if idx >= len(dr_array):
        return dr_array[-1]
    return dr_array[idx]


# -----------------------------------------------------------------------------
# Shared voting helpers
# -----------------------------------------------------------------------------
def _build_rng(tie_break, random_seed):
    if tie_break == "random":
        return random.Random(random_seed)
    return None


def _choose_from_candidates(candidates, tie_break="deterministic", rng=None):
    if not candidates:
        raise ValueError("No candidates provided.")
    if tie_break == "random":
        if rng is None:
            rng = random.Random()
        return rng.choice(list(candidates))
    return sorted(candidates)[0]


def _argmax_project(scores, tie_break="deterministic", rng=None):
    best_value = max(scores.values())
    candidates = [p for p, v in scores.items() if np.isclose(v, best_value)]
    return _choose_from_candidates(candidates, tie_break=tie_break, rng=rng)


def _extract_and_validate_credences(custom_worldviews, require_sum_to_one=False, tolerance=1e-6):
    credences = []
    for idx, worldview in enumerate(custom_worldviews):
        if "credence" not in worldview:
            raise ValueError(f"Worldview at index {idx} is missing 'credence'.")
        credence = float(worldview["credence"])
        if credence < 0:
            name = worldview.get("name", f"worldview_{idx}")
            raise ValueError(f"Credence for worldview '{name}' must be non-negative.")
        credences.append(credence)

    total = float(sum(credences))
    if require_sum_to_one and not np.isclose(total, 1.0, atol=tolerance):
        raise ValueError(f"Worldview credences must sum to 1.0. Got {total:.12f}.")

    return credences, total


def _normalize_credences(custom_worldviews):
    credences, total = _extract_and_validate_credences(custom_worldviews, require_sum_to_one=False)
    if total <= 0:
        return [0.0 for _ in credences]
    return [c / total for c in credences]


def _compute_worldview_marginal_values(data, funding, worldview):
    base_values = calculate_all_projects(
        data,
        worldview["moral_weights"],
        worldview["discount_factors"],
        worldview["risk_profile"],
    )
    adjusted_values = adjust_for_extinction_risk(base_values, data, worldview["p_extinction"])
    return {
        project_id: adjusted_values[project_id]
        * get_diminishing_returns_factor(data, project_id, funding[project_id])
        for project_id in data
    }


def _compute_all_worldview_marginal_values(data, funding, custom_worldviews):
    return [_compute_worldview_marginal_values(data, funding, worldview) for worldview in custom_worldviews]


def _build_project_ranking(project_scores):
    return sorted(project_scores.keys(), key=lambda p: (-project_scores[p], p))


def _resolve_msa_worldview_type(worldview, worldview_types=None):
    explicit = str(worldview.get("theory_type", "")).strip().lower()
    if explicit in {"binary", "cardinal"}:
        return explicit

    name = worldview.get("name", "")
    if worldview_types and name in worldview_types:
        mapped = str(worldview_types[name]).strip().lower()
        if mapped in {"binary", "cardinal"}:
            return mapped

    if name in MSA_DEFAULT_BINARY_WORLDVIEWS:
        return "binary"
    return "cardinal"


def _hhi(funding):
    total = sum(funding.values())
    if total <= 0:
        return 0.0
    shares = [amount / total for amount in funding.values()]
    return float(sum(share * share for share in shares))


# -----------------------------------------------------------------------------
# MET similarity helpers (adapted from met_sim_utils.py)
# -----------------------------------------------------------------------------
def calculate_pairwise_similarities(worldviews, projects):
    n_worldviews = len(worldviews)
    pearson_matrix = np.zeros((n_worldviews, n_worldviews))
    rank_matrix = np.zeros((n_worldviews, n_worldviews))

    for i, worldview_i in enumerate(worldviews):
        for j, worldview_j in enumerate(worldviews):
            if i == j:
                pearson_matrix[i, j] = 1.0
                rank_matrix[i, j] = 1.0
            else:
                values_i = [worldview_i.evaluate(project) for project in projects]
                values_j = [worldview_j.evaluate(project) for project in projects]

                if len(set(values_i)) <= 1 or len(set(values_j)) <= 1:
                    pearson_corr = 0.0
                else:
                    pearson_corr, _ = pearsonr(values_i, values_j)
                    if np.isnan(pearson_corr):
                        pearson_corr = 0.0
                pearson_matrix[i, j] = (pearson_corr + 1) / 2

                rank_corr, _ = spearmanr(values_i, values_j)
                if np.isnan(rank_corr):
                    rank_corr = 0.0
                rank_matrix[i, j] = (rank_corr + 1) / 2

    return pearson_matrix, rank_matrix


def embed_worldviews_in_2d_space(pearson_matrix, rank_matrix):
    n_worldviews = pearson_matrix.shape[0]
    if n_worldviews == 1:
        return np.array([[0.0, 0.0]])

    distance_matrix = np.zeros((n_worldviews, n_worldviews))
    for i in range(n_worldviews):
        for j in range(n_worldviews):
            pearson_dist = 1 - pearson_matrix[i, j]
            rank_dist = 1 - rank_matrix[i, j]
            distance_matrix[i, j] = np.sqrt(pearson_dist**2 + rank_dist**2)

    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)
    positions = mds.fit_transform(distance_matrix)
    return positions


def calculate_weighted_centroid(positions, weights):
    if np.sum(weights) == 0:
        return np.array([0.0, 0.0])
    return np.average(positions, axis=0, weights=weights)


def find_closest_worldview(worldview_positions, target_point):
    distances = [euclidean(pos, target_point) for pos in worldview_positions]
    return int(np.argmin(distances))


# -----------------------------------------------------------------------------
# MSA helpers (adapted from multi_stage_aggregation.py)
# -----------------------------------------------------------------------------
@dataclass
class MoralTheory:
    name: str
    intervention_values: Dict[str, float]

    def value_of(self, intervention: str) -> float:
        return float(self.intervention_values.get(intervention, 0.0))


def mec_aggregate_cardinal_theories(interventions, cardinal_theories, credence_distribution):
    if not interventions:
        raise ValueError("interventions must not be empty")

    intervention_scores = {}
    for intervention in interventions:
        score = 0.0
        for theory in cardinal_theories:
            score += credence_distribution.get(theory.name, 0.0) * theory.value_of(intervention)
        intervention_scores[intervention] = score

    best_intervention = max(interventions, key=lambda i: intervention_scores[i])
    return best_intervention, intervention_scores


# -----------------------------------------------------------------------------
# Voting methods
# -----------------------------------------------------------------------------
def vote_credence_weighted_custom(data, funding, increment, custom_worldviews):
    allocations = {p: 0 for p in data}
    credences, total_credence = _extract_and_validate_credences(custom_worldviews, require_sum_to_one=False)

    if not (
        np.isclose(total_credence, 1.0, atol=1e-6)
        or np.isclose(total_credence, 0.0, atol=1e-12)
    ):
        raise ValueError(f"Worldview credences must sum to 1.0 (or all zero). Got {total_credence:.12f}.")

    if np.isclose(total_credence, 0.0, atol=1e-12):
        return allocations

    rng = _build_rng(AGGREGATION_DEFAULTS["tie_break"], None)
    for worldview, credence in zip(custom_worldviews, credences):
        share = credence * increment
        marginal_values = _compute_worldview_marginal_values(data, funding, worldview)
        best_project = _argmax_project(marginal_values, tie_break="deterministic", rng=rng)
        allocations[best_project] += share

    return allocations


def vote_my_favorite_theory(
    data,
    funding,
    increment,
    results=None,
    worldviews=None,
    custom_worldviews=None,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)

    if custom_worldviews is not None:
        if not custom_worldviews:
            return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

        credences = _normalize_credences(custom_worldviews)
        if np.isclose(sum(credences), 0.0):
            return (allocations, {"strategy": "no_positive_credence"}) if return_debug else allocations

        best_idx = int(np.argmax(credences))
        selected_worldview = custom_worldviews[best_idx]
        marginal_values = _compute_worldview_marginal_values(data, funding, selected_worldview)
        best_project = _argmax_project(marginal_values, tie_break=tie_break, rng=rng)
        allocations[best_project] = increment

        if return_debug:
            return allocations, {
                "strategy": "custom_worldviews",
                "selected_worldview": selected_worldview.get("name", f"worldview_{best_idx}"),
                "selected_project": best_project,
            }
        return allocations

    if worldviews is None or results is None:
        raise ValueError(
            "vote_my_favorite_theory requires either custom_worldviews or legacy results + worldviews."
        )

    if not worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    best_wv = max(worldviews, key=lambda worldview: worldview["credence"])
    base_values = results[best_wv["result_idx"]]["project_values"]
    marginal_values = {
        project_id: base_values[project_id]
        * get_diminishing_returns_factor(data, project_id, funding[project_id])
        for project_id in data
    }
    best_project = _argmax_project(marginal_values, tie_break=tie_break, rng=rng)
    allocations[best_project] = increment

    if return_debug:
        return allocations, {
            "strategy": "legacy_precomputed",
            "selected_worldview": best_wv.get("name", "legacy_worldview"),
            "selected_project": best_project,
        }
    return allocations


def vote_mec(
    data,
    funding,
    increment,
    q1_cred=None,
    q2_cred=None,
    q3_cred=None,
    q4_cred=None,
    q5_cred=None,
    q6_cred=None,
    q7_cred=None,
    q1_daly_weights=None,
    q2_income_weights=None,
    q3_chicken_multipliers=None,
    q4_shrimp_multipliers=None,
    q5_discount_factors=None,
    q7_extinction_probs=None,
    build_moral_weights_fn=None,
    custom_worldviews=None,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)

    if custom_worldviews is not None:
        if not custom_worldviews:
            return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

        credences = _normalize_credences(custom_worldviews)
        if np.isclose(sum(credences), 0.0):
            return (allocations, {"strategy": "no_positive_credence"}) if return_debug else allocations

        worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
        expected_scores = {
            project_id: sum(
                credences[idx] * worldview_scores[idx][project_id]
                for idx in range(len(custom_worldviews))
            )
            for project_id in data
        }
        best_project = _argmax_project(expected_scores, tie_break=tie_break, rng=rng)
        allocations[best_project] = increment

        if return_debug:
            return allocations, {
                "strategy": "custom_worldviews",
                "expected_scores": expected_scores,
                "selected_project": best_project,
            }
        return allocations

    required_legacy = {
        "q1_cred": q1_cred,
        "q2_cred": q2_cred,
        "q3_cred": q3_cred,
        "q4_cred": q4_cred,
        "q5_cred": q5_cred,
        "q6_cred": q6_cred,
        "q7_cred": q7_cred,
        "q1_daly_weights": q1_daly_weights,
        "q2_income_weights": q2_income_weights,
        "q3_chicken_multipliers": q3_chicken_multipliers,
        "q4_shrimp_multipliers": q4_shrimp_multipliers,
        "q5_discount_factors": q5_discount_factors,
        "q7_extinction_probs": q7_extinction_probs,
        "build_moral_weights_fn": build_moral_weights_fn,
    }
    missing = [name for name, value in required_legacy.items() if value is None]
    if missing:
        raise ValueError("vote_mec legacy interface is missing required parameters: " + ", ".join(missing))

    avg_q1 = sum(c * v for c, v in zip(q1_cred, q1_daly_weights))
    avg_q2 = sum(c * v for c, v in zip(q2_cred, q2_income_weights))
    avg_q3 = sum(c * v for c, v in zip(q3_cred, q3_chicken_multipliers))
    avg_q4 = sum(c * v for c, v in zip(q4_cred, q4_shrimp_multipliers))
    avg_q5 = [
        sum(c * v for c, v in zip(q5_cred, [q5_discount_factors[i][t] for i in range(4)]))
        for t in range(6)
    ]
    avg_q7 = sum(c * v for c, v in zip(q7_cred, q7_extinction_probs))
    moral_weights = build_moral_weights_fn(avg_q1, avg_q2, avg_q3, avg_q4)

    for risk_idx, risk_credence in enumerate(q6_cred):
        if risk_credence == 0:
            continue

        base_values = calculate_all_projects(data, moral_weights, avg_q5, risk_idx)
        adjusted_values = adjust_for_extinction_risk(base_values, data, avg_q7)

        marginal_values = {
            p: adjusted_values[p] * get_diminishing_returns_factor(data, p, funding[p])
            for p in data
        }
        best_project = _argmax_project(marginal_values, tie_break=tie_break, rng=rng)
        allocations[best_project] += risk_credence * increment

    if return_debug:
        return allocations, {"strategy": "legacy_quiz_inputs"}
    return allocations


def vote_met(
    data,
    funding,
    increment,
    custom_worldviews,
    met_threshold=None,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    if not custom_worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    threshold = AGGREGATION_DEFAULTS["met_threshold"] if met_threshold is None else met_threshold
    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)

    worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
    credences = _normalize_credences(custom_worldviews)
    max_idx = int(np.argmax(credences))
    max_credence = credences[max_idx]

    strategy = "favorite_theory"
    selected_idx = max_idx

    if max_credence < threshold:

        class _WorldviewAdapter:
            def __init__(self, scores):
                self._scores = scores

            def evaluate(self, project_id):
                return self._scores[project_id]

        projects = list(data.keys())
        adapters = [_WorldviewAdapter(scores) for scores in worldview_scores]
        pearson_matrix, rank_matrix = calculate_pairwise_similarities(adapters, projects)
        positions = embed_worldviews_in_2d_space(pearson_matrix, rank_matrix)
        centroid = calculate_weighted_centroid(positions, np.array(credences))
        selected_idx = find_closest_worldview(positions, centroid)
        strategy = "similarity_centroid"

    selected_scores = worldview_scores[selected_idx]
    best_project = _argmax_project(selected_scores, tie_break=tie_break, rng=rng)
    allocations[best_project] = increment

    if return_debug:
        return allocations, {
            "strategy": strategy,
            "threshold": threshold,
            "max_credence": max_credence,
            "selected_worldview": custom_worldviews[selected_idx].get("name", f"worldview_{selected_idx}"),
            "selected_project": best_project,
        }
    return allocations


def _nash_disagreement_utilities(worldview_scores, credences, disagreement_point, tie_break="deterministic", rng=None):
    n_worldviews = len(worldview_scores)
    projects = list(worldview_scores[0].keys()) if worldview_scores else []
    best_projects = [_argmax_project(scores, tie_break=tie_break, rng=rng) for scores in worldview_scores]

    if disagreement_point == "zero_spending":
        return [0.0 for _ in range(n_worldviews)]

    if disagreement_point == "anti_utopia":
        return [min(scores[p] for p in projects) for scores in worldview_scores]

    if disagreement_point == "random_dictator":
        utilities = []
        for i in range(n_worldviews):
            baseline = 0.0
            for j in range(n_worldviews):
                baseline += credences[j] * worldview_scores[i][best_projects[j]]
            utilities.append(baseline)
        return utilities

    if disagreement_point == "exclusionary_proportional_split":
        utilities = []
        for i in range(n_worldviews):
            own_credence = credences[i]
            if own_credence >= 1.0:
                utilities.append(0.0)
                continue
            baseline = 0.0
            denominator = 1.0 - own_credence
            for j in range(n_worldviews):
                if j == i:
                    continue
                baseline += (credences[j] / denominator) * worldview_scores[i][best_projects[j]]
            utilities.append(baseline)
        return utilities

    raise ValueError(
        "Unknown disagreement_point. Use one of: zero_spending, anti_utopia, random_dictator, exclusionary_proportional_split."
    )


def vote_nash_bargaining(
    data,
    funding,
    increment,
    custom_worldviews,
    disagreement_point=None,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    if not custom_worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    disagreement_point = (
        AGGREGATION_DEFAULTS["nash_disagreement_point"]
        if disagreement_point is None
        else disagreement_point
    )
    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)

    worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
    credences = _normalize_credences(custom_worldviews)
    projects = list(data.keys())

    disagreement_utilities = _nash_disagreement_utilities(
        worldview_scores,
        credences,
        disagreement_point,
        tie_break=tie_break,
        rng=rng,
    )

    feasible_scores = {}
    fallback_scores = {}
    for project_id in projects:
        gains = [
            worldview_scores[i][project_id] - disagreement_utilities[i]
            for i in range(len(custom_worldviews))
        ]
        if all(g >= -1e-12 for g in gains):
            feasible_scores[project_id] = float(np.prod([max(g, 0.0) for g in gains]))
        fallback_scores[project_id] = float(sum(gains))

    if feasible_scores:
        best_value = max(feasible_scores.values())
        candidates = [p for p, score in feasible_scores.items() if np.isclose(score, best_value)]
        objective_used = "nash_product"
        objective_scores = feasible_scores
    else:
        best_value = max(fallback_scores.values())
        candidates = [p for p, score in fallback_scores.items() if np.isclose(score, best_value)]
        objective_used = "sum_gains_fallback"
        objective_scores = fallback_scores

    selected_project = _choose_from_candidates(candidates, tie_break=tie_break, rng=rng)
    allocations[selected_project] = increment

    if return_debug:
        return allocations, {
            "disagreement_point": disagreement_point,
            "objective": objective_used,
            "objective_scores": objective_scores,
            "disagreement_utilities": disagreement_utilities,
            "selected_project": selected_project,
        }
    return allocations


def vote_msa(
    data,
    funding,
    increment,
    custom_worldviews,
    worldview_types=None,
    cardinal_permissibility_mode=None,
    cardinal_top_k=None,
    cardinal_within_percent=None,
    binary_permissibility_threshold=None,
    no_permissible_action="stop",
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    if not custom_worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)
    cardinal_permissibility_mode = (
        AGGREGATION_DEFAULTS["msa_permissibility_mode"]
        if cardinal_permissibility_mode is None
        else cardinal_permissibility_mode
    )
    cardinal_top_k = AGGREGATION_DEFAULTS["msa_top_k"] if cardinal_top_k is None else cardinal_top_k
    cardinal_within_percent = (
        AGGREGATION_DEFAULTS["msa_within_percent"]
        if cardinal_within_percent is None
        else cardinal_within_percent
    )
    binary_permissibility_threshold = (
        AGGREGATION_DEFAULTS["msa_binary_threshold"]
        if binary_permissibility_threshold is None
        else binary_permissibility_threshold
    )

    worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
    credences = _normalize_credences(custom_worldviews)
    projects = list(data.keys())

    cardinal_indices = []
    binary_indices = []
    for idx, worldview in enumerate(custom_worldviews):
        worldview_type = _resolve_msa_worldview_type(worldview, worldview_types=worldview_types)
        if worldview_type == "binary":
            binary_indices.append(idx)
        else:
            cardinal_indices.append(idx)

    cardinal_cluster_credence = sum(credences[idx] for idx in cardinal_indices)
    mec_scores = {project_id: 0.0 for project_id in projects}
    cardinal_best = None

    if cardinal_indices:
        cardinal_theories = []
        credence_distribution = {}
        for idx in cardinal_indices:
            name = custom_worldviews[idx].get("name", f"worldview_{idx}")
            theory_name = f"{name}_{idx}"
            cardinal_theories.append(MoralTheory(theory_name, worldview_scores[idx]))
            credence_distribution[theory_name] = credences[idx]

        cardinal_best, mec_scores = mec_aggregate_cardinal_theories(
            projects, cardinal_theories, credence_distribution
        )

    cardinal_permissible = set()
    threshold_score = None
    if cardinal_indices:
        if cardinal_permissibility_mode == "winner_take_all":
            cardinal_permissible = {cardinal_best}
        elif cardinal_permissibility_mode == "top_k":
            k = max(1, int(cardinal_top_k))
            ranked = sorted(projects, key=lambda p: (-mec_scores[p], p))
            cardinal_permissible = set(ranked[: min(k, len(ranked))])
        elif cardinal_permissibility_mode == "within_percent":
            if cardinal_within_percent < 0:
                raise ValueError("cardinal_within_percent must be >= 0.")
            best_score = mec_scores[cardinal_best]
            threshold_score = best_score - abs(best_score) * float(cardinal_within_percent)
            cardinal_permissible = {
                p for p in projects if mec_scores[p] >= threshold_score - 1e-12
            }
        else:
            raise ValueError(
                "Unknown cardinal_permissibility_mode. Use one of: winner_take_all, top_k, within_percent."
            )

    vote_tallies = {project_id: 0.0 for project_id in projects}

    for project_id in cardinal_permissible:
        vote_tallies[project_id] += cardinal_cluster_credence

    for idx in binary_indices:
        worldview_credence = credences[idx]
        scores = worldview_scores[idx]
        for project_id in projects:
            if scores[project_id] > binary_permissibility_threshold:
                vote_tallies[project_id] += worldview_credence

    max_tally = max(vote_tallies.values()) if vote_tallies else 0.0
    if max_tally <= 0.5:
        if no_permissible_action == "stop":
            stop_signal = {
                "__stop__": True,
                "__reason__": "No intervention exceeded 50% permissibility.",
            }
            debug = {
                "vote_tallies": vote_tallies,
                "mec_scores": mec_scores,
                "cardinal_permissible": sorted(cardinal_permissible),
            }
            return (stop_signal, debug) if return_debug else stop_signal

        if no_permissible_action == "fallback_mec":
            if cardinal_indices:
                selected_project = _argmax_project(mec_scores, tie_break=tie_break, rng=rng)
            else:
                weighted_scores = {
                    project_id: sum(
                        credences[idx] * worldview_scores[idx][project_id]
                        for idx in range(len(custom_worldviews))
                    )
                    for project_id in projects
                }
                selected_project = _argmax_project(weighted_scores, tie_break=tie_break, rng=rng)
            allocations[selected_project] = increment
            if return_debug:
                return allocations, {
                    "fallback_used": True,
                    "selected_project": selected_project,
                    "vote_tallies": vote_tallies,
                    "mec_scores": mec_scores,
                }
            return allocations

        raise ValueError("Unknown no_permissible_action. Use stop or fallback_mec.")

    winners = [project_id for project_id, tally in vote_tallies.items() if np.isclose(tally, max_tally)]
    selected_project = _choose_from_candidates(winners, tie_break=tie_break, rng=rng)
    allocations[selected_project] = increment

    if return_debug:
        return allocations, {
            "selected_project": selected_project,
            "vote_tallies": vote_tallies,
            "mec_scores": mec_scores,
            "cardinal_permissible": sorted(cardinal_permissible),
            "cardinal_permissibility_mode": cardinal_permissibility_mode,
            "threshold_score": threshold_score,
        }
    return allocations


def vote_borda(
    data,
    funding,
    increment,
    custom_worldviews,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    if not custom_worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)
    worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
    credences = _normalize_credences(custom_worldviews)
    projects = list(data.keys())
    n_projects = len(projects)

    borda_scores = {project_id: 0.0 for project_id in projects}
    for idx, scores in enumerate(worldview_scores):
        ranking = _build_project_ranking(scores)
        for rank_idx, project_id in enumerate(ranking):
            points = (n_projects - 1) - rank_idx
            borda_scores[project_id] += credences[idx] * points

    best_value = max(borda_scores.values())
    winners = [project_id for project_id, score in borda_scores.items() if np.isclose(score, best_value)]
    selected_project = _choose_from_candidates(winners, tie_break=tie_break, rng=rng)
    allocations[selected_project] = increment

    if return_debug:
        return allocations, {"borda_scores": borda_scores, "selected_project": selected_project}
    return allocations


def vote_split_cycle(
    data,
    funding,
    increment,
    custom_worldviews,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    if not custom_worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)
    worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
    credences = _normalize_credences(custom_worldviews)
    projects = list(data.keys())

    preferences = {a: {b: 0.0 for b in projects} for a in projects}

    for idx, scores in enumerate(worldview_scores):
        weight = credences[idx]
        for i, project_a in enumerate(projects):
            for j in range(i + 1, len(projects)):
                project_b = projects[j]
                if scores[project_a] > scores[project_b]:
                    preferences[project_a][project_b] += weight
                elif scores[project_b] > scores[project_a]:
                    preferences[project_b][project_a] += weight

    margins = {
        a: {b: preferences[a][b] - preferences[b][a] for b in projects}
        for a in projects
    }

    neg_inf = float("-inf")
    strongest_path = {
        a: {b: (margins[a][b] if margins[a][b] > 0 else neg_inf) for b in projects}
        for a in projects
    }
    for p in projects:
        strongest_path[p][p] = 0.0

    for k in projects:
        for i in projects:
            if i == k:
                continue
            for j in projects:
                if i == j or j == k:
                    continue
                via_k = min(strongest_path[i][k], strongest_path[k][j])
                if via_k > strongest_path[i][j]:
                    strongest_path[i][j] = via_k

    defeats = {
        a: {
            b: (margins[a][b] > 0 and margins[a][b] > strongest_path[b][a] + 1e-12)
            for b in projects
        }
        for a in projects
    }

    unbeaten = [
        candidate
        for candidate in projects
        if not any(defeats[other][candidate] for other in projects if other != candidate)
    ]

    if unbeaten:
        winners = unbeaten
    else:
        net_scores = {
            candidate: sum(margins[candidate][other] for other in projects if other != candidate)
            for candidate in projects
        }
        best_net = max(net_scores.values())
        winners = [p for p, score in net_scores.items() if np.isclose(score, best_net)]

    selected_project = _choose_from_candidates(winners, tie_break=tie_break, rng=rng)
    allocations[selected_project] = increment

    if return_debug:
        return allocations, {
            "margins": margins,
            "strongest_path": strongest_path,
            "defeats": defeats,
            "selected_project": selected_project,
        }
    return allocations


def vote_lexicographic_maximin(
    data,
    funding,
    increment,
    custom_worldviews,
    tie_break=None,
    random_seed=None,
    return_debug=False,
):
    allocations = {p: 0 for p in data}
    if not custom_worldviews:
        return (allocations, {"strategy": "no_worldviews"}) if return_debug else allocations

    tie_break = AGGREGATION_DEFAULTS["tie_break"] if tie_break is None else tie_break
    rng = _build_rng(tie_break, random_seed)
    worldview_scores = _compute_all_worldview_marginal_values(data, funding, custom_worldviews)
    credences = _normalize_credences(custom_worldviews)
    projects = list(data.keys())

    vectors = {}
    for project_id in projects:
        weighted_utilities = [
            credences[idx] * worldview_scores[idx][project_id]
            for idx in range(len(custom_worldviews))
        ]
        vectors[project_id] = tuple(sorted(weighted_utilities))

    best_vector = max(vectors.values())
    winners = [project_id for project_id, vector in vectors.items() if vector == best_vector]
    selected_project = _choose_from_candidates(winners, tie_break=tie_break, rng=rng)
    allocations[selected_project] = increment

    if return_debug:
        return allocations, {"vectors": vectors, "selected_project": selected_project}
    return allocations


# -----------------------------------------------------------------------------
# Iteration loop and display helpers
# -----------------------------------------------------------------------------
def allocate_budget(data, voting_method, total_budget, increment_size=None, **kwargs):
    if increment_size is None:
        increment_size = INCREMENT_SIZE

    funding = {project_id: 0 for project_id in data}
    history = []
    remaining = total_budget

    while remaining > 0:
        increment = min(increment_size, remaining)
        vote_output = voting_method(data, funding, increment, **kwargs)

        metadata = {}
        if (
            isinstance(vote_output, tuple)
            and len(vote_output) == 2
            and isinstance(vote_output[0], dict)
        ):
            allocations, metadata = vote_output
            if not isinstance(metadata, dict):
                metadata = {"raw_metadata": metadata}
        else:
            allocations = vote_output

        if not isinstance(allocations, dict):
            raise TypeError("Voting methods must return a dict of allocations.")

        if allocations.get("__stop__", False):
            stop_entry = {
                "iteration": len(history),
                "allocations": {project_id: 0 for project_id in data},
                "stopped": True,
                "remaining_budget": remaining,
            }
            if "__reason__" in allocations:
                stop_entry["reason"] = allocations["__reason__"]
            if metadata:
                stop_entry["meta"] = metadata
            history.append(stop_entry)
            break

        for project_id in data:
            funding[project_id] += allocations.get(project_id, 0)

        history_entry = {
            "iteration": len(history),
            "allocations": {project_id: allocations.get(project_id, 0) for project_id in data},
        }
        if metadata:
            history_entry["meta"] = metadata
        history.append(history_entry)
        remaining -= increment

    return {"funding": funding, "history": history}


def show_allocation(allocation, data):
    funding = allocation["funding"]
    total = sum(funding.values())
    print("=" * 60)
    print(f"BUDGET ALLOCATION (total: ${total:,.1f}M)")
    print("=" * 60)
    print(f"{'Project':<24} {'Allocated':>10} {'% budget':>10}")
    print("-" * 60)
    for project_id, amount in sorted(funding.items(), key=lambda x: x[1], reverse=True):
        pct = (amount / total * 100) if total > 0 else 0
        print(f"{project_id:<24} ${amount:>7.1f}M {pct:>8.1f}%")


def funding_dataframe(allocation):
    funding = allocation["funding"]
    total = sum(funding.values())
    rows = []
    for project_id, amount in sorted(funding.items(), key=lambda x: x[1], reverse=True):
        rows.append(
            {
                "project_id": project_id,
                "funding": amount,
                "percent": (amount / total * 100) if total > 0 else 0.0,
            }
        )
    return pd.DataFrame(rows)


def _show_df(df):
    try:
        from IPython.display import display

        display(df)
    except Exception:
        print(df.to_string(index=False))


METHOD_REGISTRY = {
    "credence_weighted": vote_credence_weighted_custom,
    "my_favorite_theory": vote_my_favorite_theory,
    "mec": vote_mec,
    "met": vote_met,
    "nash_bargaining": vote_nash_bargaining,
    "msa": vote_msa,
    "borda": vote_borda,
    "split_cycle": vote_split_cycle,
    "lexicographic_maximin": vote_lexicographic_maximin,
}

print("Loaded standalone Donor Compass snapshot.")
print("Available methods:", list(METHOD_REGISTRY.keys()))

Loaded standalone Donor Compass snapshot.
Available methods: ['credence_weighted', 'my_favorite_theory', 'mec', 'met', 'nash_bargaining', 'msa', 'borda', 'split_cycle', 'lexicographic_maximin']


## Demo Run (Single Method)

Edit `SELECTED_METHOD`, budget, and method-specific options in the next cell, then run the execution cell.

In [9]:
from copy import deepcopy

# Core run settings
SELECTED_METHOD = "credence_weighted"  # change to any key in METHOD_REGISTRY
TOTAL_BUDGET = 100
INCREMENT_SIZE = 10

# Start from example worldviews and edit values as desired
CUSTOM_WORLDVIEWS = deepcopy(EXAMPLE_CUSTOM_WORLDVIEWS)

# Global tie settings for methods that support it
TIE_BREAK = "deterministic"  # deterministic | random
RANDOM_SEED = 42

# Method-specific settings
MET_THRESHOLD = 0.50
NASH_DISAGREEMENT_POINT = "zero_spending"  # zero_spending | anti_utopia | random_dictator | exclusionary_proportional_split
MSA_CARDINAL_MODE = "winner_take_all"      # winner_take_all | top_k | within_percent
MSA_TOP_K = 2
MSA_WITHIN_PERCENT = 0.10
MSA_BINARY_THRESHOLD = 0.0
MSA_NO_PERMISSIBLE_ACTION = "fallback_mec" # stop | fallback_mec


def build_method_kwargs(method_name, custom_worldviews):
    kwargs = {"custom_worldviews": custom_worldviews}

    if method_name == "met":
        kwargs.update({
            "met_threshold": MET_THRESHOLD,
            "tie_break": TIE_BREAK,
            "random_seed": RANDOM_SEED,
        })
    elif method_name == "nash_bargaining":
        kwargs.update({
            "disagreement_point": NASH_DISAGREEMENT_POINT,
            "tie_break": TIE_BREAK,
            "random_seed": RANDOM_SEED,
        })
    elif method_name == "msa":
        kwargs.update({
            "cardinal_permissibility_mode": MSA_CARDINAL_MODE,
            "cardinal_top_k": MSA_TOP_K,
            "cardinal_within_percent": MSA_WITHIN_PERCENT,
            "binary_permissibility_threshold": MSA_BINARY_THRESHOLD,
            "no_permissible_action": MSA_NO_PERMISSIBLE_ACTION,
            "tie_break": TIE_BREAK,
            "random_seed": RANDOM_SEED,
        })
    elif method_name in {
        "my_favorite_theory",
        "mec",
        "borda",
        "split_cycle",
        "lexicographic_maximin",
    }:
        kwargs.update({"tie_break": TIE_BREAK, "random_seed": RANDOM_SEED})

    return kwargs


print("Selected method:", SELECTED_METHOD)
print("Available methods:", list(METHOD_REGISTRY.keys()))

Selected method: credence_weighted
Available methods: ['credence_weighted', 'my_favorite_theory', 'mec', 'met', 'nash_bargaining', 'msa', 'borda', 'split_cycle', 'lexicographic_maximin']


In [10]:
# Run the selected method
if SELECTED_METHOD not in METHOD_REGISTRY:
    raise ValueError(f"Unknown method: {SELECTED_METHOD}")

method_fn = METHOD_REGISTRY[SELECTED_METHOD]
method_kwargs = build_method_kwargs(SELECTED_METHOD, deepcopy(CUSTOM_WORLDVIEWS))

allocation = allocate_budget(
    DEFAULT_PROJECT_DATA,
    method_fn,
    total_budget=TOTAL_BUDGET,
    increment_size=INCREMENT_SIZE,
    **method_kwargs,
)

show_allocation(allocation, DEFAULT_PROJECT_DATA)

print("\nFunding table:")
_show_df(funding_dataframe(allocation))

if allocation["history"] and allocation["history"][-1].get("stopped"):
    last = allocation["history"][-1]
    print("\nAllocation stopped early.")
    print("Reason:", last.get("reason"))
    print("Remaining budget:", last.get("remaining_budget"))

print("\nHistory preview (first 5 iterations):")
for entry in allocation["history"][:5]:
    print(entry)

BUDGET ALLOCATION (total: $100.0M)
Project                   Allocated   % budget
------------------------------------------------------------
project_human            $   50.0M     50.0%
project_chicken          $   30.0M     30.0%
project_fish             $   20.0M     20.0%

Funding table:


,project_id,funding,percent
0,project_human,50.0,50.0
1,project_chicken,30.0,30.0
2,project_fish,20.0,20.0



History preview (first 5 iterations):
{'iteration': 0, 'allocations': {'project_human': 5.0, 'project_chicken': 3.0, 'project_fish': 2.0}}
{'iteration': 1, 'allocations': {'project_human': 5.0, 'project_chicken': 3.0, 'project_fish': 2.0}}
{'iteration': 2, 'allocations': {'project_human': 5.0, 'project_chicken': 3.0, 'project_fish': 2.0}}
{'iteration': 3, 'allocations': {'project_human': 5.0, 'project_chicken': 3.0, 'project_fish': 2.0}}
{'iteration': 4, 'allocations': {'project_human': 5.0, 'project_chicken': 3.0, 'project_fish': 2.0}}


## Compare All Methods On The Same Inputs

This runs every aggregation method with the same worldviews and budget, then reports top project and concentration (HHI).

In [11]:
comparison_rows = []
for method_name, method_fn in METHOD_REGISTRY.items():
    kwargs = build_method_kwargs(method_name, deepcopy(CUSTOM_WORLDVIEWS))
    result = allocate_budget(
        DEFAULT_PROJECT_DATA,
        method_fn,
        total_budget=TOTAL_BUDGET,
        increment_size=INCREMENT_SIZE,
        **kwargs,
    )
    funding = result["funding"]
    top_project = max(funding, key=funding.get)
    comparison_rows.append(
        {
            "method": method_name,
            "top_project": top_project,
            "hhi": _hhi(funding),
            "funding": funding,
            "stopped_early": bool(result["history"] and result["history"][-1].get("stopped")),
        }
    )

comparison_df = pd.DataFrame(comparison_rows).sort_values("hhi")
_show_df(comparison_df)

print("\nFunding by method:")
for row in comparison_rows:
    print(f"- {row['method']}: {row['funding']}")

,method,top_project,hhi,funding,stopped_early
0,credence_weighted,project_human,0.38,"{'project_human': 50.0, 'project_chicken': 30....",False
4,nash_bargaining,project_chicken,0.38,"{'project_human': 20, 'project_chicken': 50, '...",False
8,lexicographic_maximin,project_fish,0.38,"{'project_human': 20, 'project_chicken': 30, '...",False
2,mec,project_chicken,0.52,"{'project_human': 40, 'project_chicken': 60, '...",False
5,msa,project_chicken,0.52,"{'project_human': 40, 'project_chicken': 60, '...",False
6,borda,project_chicken,0.58,"{'project_human': 30, 'project_chicken': 70, '...",False
7,split_cycle,project_chicken,0.58,"{'project_human': 30, 'project_chicken': 70, '...",False
1,my_favorite_theory,project_human,1.00,"{'project_human': 100, 'project_chicken': 0, '...",False
3,met,project_human,1.00,"{'project_human': 100, 'project_chicken': 0, '...",False



Funding by method:
- credence_weighted: {'project_human': 50.0, 'project_chicken': 30.0, 'project_fish': 20.0}
- my_favorite_theory: {'project_human': 100, 'project_chicken': 0, 'project_fish': 0}
- mec: {'project_human': 40, 'project_chicken': 60, 'project_fish': 0}
- met: {'project_human': 100, 'project_chicken': 0, 'project_fish': 0}
- nash_bargaining: {'project_human': 20, 'project_chicken': 50, 'project_fish': 30}
- msa: {'project_human': 40, 'project_chicken': 60, 'project_fish': 0}
- borda: {'project_human': 30, 'project_chicken': 70, 'project_fish': 0}
- split_cycle: {'project_human': 30, 'project_chicken': 70, 'project_fish': 0}
- lexicographic_maximin: {'project_human': 20, 'project_chicken': 30, 'project_fish': 50}


## End-To-End Validation Checks

These are the only verification checks included here (no full unit-test suite).

- **Check 1: Dominant Then Saturating Project**
  - Starts with one project clearly best at first.
  - As diminishing returns kick in, methods should eventually switch away from that project.
- **Check 2: Same Credences Across Methods**
  - Runs every method on the same worldview credences and project inputs.
  - Lets you compare how concentrated/diversified each method's allocation is.

In [12]:
def _make_project(recipient_type, base_value, dr_curve):
    return {
        "tags": {"near_term_xrisk": False},
        "diminishing_returns": dr_curve,
        "effects": {
            "effect_main": {
                "recipient_type": recipient_type,
                "values": [[base_value] * 4 for _ in range(6)],
            }
        },
    }


def _worldview(name, credence, human=0.0, chickens=0.0, fish=0.0):
    return {
        "name": name,
        "credence": credence,
        "moral_weights": {
            "human_life_years": human,
            "human_ylds": 0.0,
            "human_income_doublings": 0.0,
            "chickens_birds": chickens,
            "fish": fish,
            "shrimp": 0.0,
            "non_shrimp_invertebrates": 0.0,
            "mammals": 0.0,
        },
        "discount_factors": [1.0] * 6,
        "risk_profile": 0,
        "p_extinction": 0.0,
    }


def _find_first_switch_iteration(history, dominant_project):
    for idx, entry in enumerate(history[1:], start=1):
        dominant_allocation = entry["allocations"].get(dominant_project, 0.0)
        other_allocation = sum(
            amount for project_id, amount in entry["allocations"].items() if project_id != dominant_project
        )
        if dominant_allocation < 10.0 and other_allocation > 0:
            return idx
    return None


# -----------------------------------------------------------------------------
# Check 1: one dominant project should win early,
# then eventually lose once diminishing returns saturate it.
# -----------------------------------------------------------------------------
dominance_data = {
    "project_dominant": _make_project("human_life_years", 500.0, [1.0, 0.05, 0.01, 0.01, 0.01, 0.01]),
    "project_b": _make_project("chickens_birds", 120.0, [1.0] * 6),
    "project_c": _make_project("fish", 100.0, [1.0] * 6),
}
aligned_worldviews = [_worldview("Aligned", 1.0, human=1.0, chickens=1.0, fish=1.0)]

scenario_a_rows = []
for method_name, method_fn in METHOD_REGISTRY.items():
    kwargs = {"custom_worldviews": aligned_worldviews}
    if method_name == "met":
        kwargs["met_threshold"] = 0.5
    if method_name == "nash_bargaining":
        kwargs["disagreement_point"] = "zero_spending"
    if method_name == "msa":
        kwargs.update(
            {
                "cardinal_permissibility_mode": "winner_take_all",
                "no_permissible_action": "fallback_mec",
            }
        )

    result = allocate_budget(
        dominance_data,
        method_fn,
        total_budget=60,
        increment_size=10,
        **kwargs,
    )
    history = result["history"]
    first_alloc = history[0]["allocations"]

    assert first_alloc["project_dominant"] > 0, (
        f"{method_name} failed dominant first-choice check: {first_alloc}"
    )

    switch_iter = _find_first_switch_iteration(history, "project_dominant")
    assert switch_iter is not None, (
        f"{method_name} did not switch after saturation. History: {history}"
    )

    non_dominant_total = result["funding"]["project_b"] + result["funding"]["project_c"]
    assert non_dominant_total > 0, (
        f"{method_name} never allocated to alternatives. Funding: {result['funding']}"
    )

    scenario_a_rows.append((method_name, switch_iter, dict(result["funding"])))


# -----------------------------------------------------------------------------
# Check 2: run all methods with identical credences
# and compare concentration/diversification behavior side-by-side.
# -----------------------------------------------------------------------------
comparison_data = {
    "project_human": _make_project(
        "human_life_years",
        220.0,
        [1.0, 0.8, 0.6, 0.5, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.12],
    ),
    "project_chicken": _make_project("chickens_birds", 180.0, [1.0] * 11),
    "project_fish": _make_project("fish", 140.0, [1.0] * 11),
}
shared_worldviews = [
    _worldview("HumanAnchor", 0.5, human=1.0, chickens=0.1, fish=0.05),
    _worldview("ChickenAnchor", 0.3, human=0.1, chickens=1.0, fish=0.2),
    _worldview("FishAnchor", 0.2, human=0.1, chickens=0.1, fish=1.0),
]

scenario_b_rows = []
scenario_b_by_name = {}

for method_name, method_fn in METHOD_REGISTRY.items():
    kwargs = {"custom_worldviews": shared_worldviews}
    if method_name == "met":
        kwargs["met_threshold"] = 0.5
    if method_name == "nash_bargaining":
        kwargs["disagreement_point"] = "zero_spending"
    if method_name == "msa":
        kwargs.update(
            {
                "cardinal_permissibility_mode": "winner_take_all",
                "no_permissible_action": "fallback_mec",
            }
        )

    result = allocate_budget(
        comparison_data,
        method_fn,
        total_budget=100,
        increment_size=10,
        **kwargs,
    )
    funding = result["funding"]
    row = {
        "method_name": method_name,
        "funding": dict(funding),
        "hhi": _hhi(funding),
        "top_project": max(funding, key=funding.get),
    }
    scenario_b_rows.append(row)
    scenario_b_by_name[method_name] = row

hhi_cw = scenario_b_by_name["credence_weighted"]["hhi"]
hhi_mec = scenario_b_by_name["mec"]["hhi"]
hhi_favorite = scenario_b_by_name["my_favorite_theory"]["hhi"]
assert hhi_cw < hhi_mec < hhi_favorite, (
    "Expected concentration ordering failed: "
    f"cw={hhi_cw:.4f}, mec={hhi_mec:.4f}, favorite={hhi_favorite:.4f}"
)


# -----------------------------------------------------------------------------
# Summary print
# -----------------------------------------------------------------------------
lines = []
lines.append("=" * 80)
lines.append("AGGREGATION STRESS TEST SUMMARY")
lines.append("=" * 80)
lines.append("")
lines.append("Check 1: Dominant project -> saturation switch")
lines.append("-" * 80)
for method_name, switch_iter, funding in scenario_a_rows:
    lines.append(f"{method_name:<24} | switch_iter={switch_iter:<2} | funding={funding}")

lines.append("")
lines.append("Check 2: Same credences, all methods side-by-side")
lines.append("-" * 80)
for row in scenario_b_rows:
    lines.append(
        f"{row['method_name']:<24} | top={row['top_project']:<15} | "
        f"HHI={row['hhi']:.4f} | funding={row['funding']}"
    )
lines.append("")
lines.append("Expected ordering validated: HHI(credence_weighted) < HHI(mec) < HHI(my_favorite_theory)")
lines.append("=" * 80)

print("\n" + "\n".join(lines))


AGGREGATION STRESS TEST SUMMARY

Check 1: Dominant project -> saturation switch
--------------------------------------------------------------------------------
credence_weighted        | switch_iter=1  | funding={'project_dominant': 10.0, 'project_b': 50.0, 'project_c': 0}
my_favorite_theory       | switch_iter=1  | funding={'project_dominant': 10, 'project_b': 50, 'project_c': 0}
mec                      | switch_iter=1  | funding={'project_dominant': 10, 'project_b': 50, 'project_c': 0}
met                      | switch_iter=1  | funding={'project_dominant': 10, 'project_b': 50, 'project_c': 0}
nash_bargaining          | switch_iter=1  | funding={'project_dominant': 10, 'project_b': 50, 'project_c': 0}
msa                      | switch_iter=1  | funding={'project_dominant': 10, 'project_b': 50, 'project_c': 0}
borda                    | switch_iter=1  | funding={'project_dominant': 10, 'project_b': 50, 'project_c': 0}
split_cycle              | switch_iter=1  | funding={'project_do